In [128]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from gensim.models import doc2vec
from gensim import models
from gensim.models.doc2vec import TaggedDocument

In [129]:
labels = pd.read_csv('targetlabels.txt',sep="\t",header=0,index_col=0)
    
data = pd.read_csv('data.csv',header=0,index_col=0)

labels['class'] = pd.Series(0,dtype=int)
labels['class'] = 0

## Generate labels data

### Note: there are three sample have both TCA and Resp, so maked as NaN

In [130]:
# Generate class for each gene.

for i in range(len(labels)):
    if labels.iloc[i].sum()!=-2:
        if labels.iloc[i]['TCA'] == 1:
            labels['class'].iloc[i] = 1
        elif labels.iloc[i]['Resp'] == 1:
            labels['class'].iloc[i] = 2
        elif labels.iloc[i]['Ribo'] == 1:
            labels['class'].iloc[i] = 3
        elif labels.iloc[i]['Proteas'] == 1:
            labels['class'].iloc[i] = 4
        elif labels.iloc[i]['Hist'] == 1:
            labels['class'].iloc[i] = 5
        elif labels.iloc[i]['HTH'] == 1:
            labels['class'].iloc[i] = 6
    elif labels.iloc[i].sum()==-6:
        labels['class'].iloc[i] = 0
    else: 
        labels['class'].iloc[i] = 7
    

In [131]:
# Three abnormal data

labels[(labels['TCA']==1)&(labels['Resp']==1)]

,TCA,Resp,Ribo,Proteas,Hist,HTH,class
gene,,,,,,,
YDR178W,1,1,-1,-1,-1,-1,7
YKL148C,1,1,-1,-1,-1,-1,7
YLL041C,1,1,-1,-1,-1,-1,7


In [132]:
# combine the label with the experiment data to a big dataframe
comb = pd.merge(data, labels['class'], left_index=True, right_index=True, how='outer')

In [133]:
comb.head(5)

,NAME,alpha 0,alpha 7,alpha 14,alpha 21,alpha 28,alpha 35,alpha 42,alpha 49,alpha 56,...,cold 40,cold 160,diau a,diau b,diau c,diau d,diau e,diau f,diau g,class
YAL001C,TFC3 TRANSCRIPTION TFIIIC 138 KD ...,-0.38,-0.38,-0.43,-0.06,-0.64,-0.67,-0.38,0.12,-0.32,...,0.93,0.29,-0.06,0.08,0.42,0.48,0.21,0.77,0.34,0
YAL002W,VPS8 VACUOLAR PROTEIN TARGETI UNKNOWN,-0.30,-0.09,-0.18,-0.14,-0.15,-0.45,-0.15,0.61,-0.42,...,-0.25,-0.04,-0.01,0.51,-0.04,-0.04,-0.23,0.06,0.90,0
YAL003W,EFB1 PROTEINSYNTHESIS ELONGATION FAC...,0.01,-0.56,0.25,-0.17,-0.25,-0.43,0.26,0.31,0.24,...,-1.15,-1.29,0.24,NaN,-0.04,-0.64,-0.94,-1.56,-2.32,0
YAL005C,SSA1 ER AND MITOCHONDRIAL TRA CYTOSOLIC HSP70,0.08,-0.40,-0.34,0.07,0.24,0.06,0.38,0.32,0.25,...,-0.18,0.30,0.12,0.06,0.04,-0.01,-0.12,0.01,-0.15,0
YAL010C,MDM10 MITOCHONDRIAL BIOGENESIS (PUTATIVE) COM...,-0.20,-0.01,-0.01,-0.36,-0.60,-0.36,-0.09,-0.29,-0.15,...,-0.34,-0.01,-1.18,-0.10,0.04,-0.89,-0.92,0.37,-0.18,0


In [134]:
# label distribution
labels.groupby('class').count()

,TCA,Resp,Ribo,Proteas,Hist,HTH
class,,,,,,
0,2240,2240,2240,2240,2240,2240
1,14,14,14,14,14,14
2,27,27,27,27,27,27
3,121,121,121,121,121,121
4,35,35,35,35,35,35
5,11,11,11,11,11,11
6,16,16,16,16,16,16
7,3,3,3,3,3,3


### class
From TCA to HTH, 1 to 6 

unlabel is 0

-2 is the three strange data

### imputer for missing data, using mean

In [135]:

X = comb[comb['class']!=7].iloc[:,1:-1]
y = comb[comb['class']!=7].iloc[:,-1]
alphaX= comb[comb['class']!=7].iloc[:,1:19]
eluX= comb[comb['class']!=7].iloc[:,19:33]
edcX= comb[comb['class']!=7].iloc[:,33:48]
spotX= comb[comb['class']!=7].iloc[:,48:57]
spoX= comb[comb['class']!=7].iloc[:,57:59]
heatX= comb[comb['class']!=7].iloc[:,59:65]
dttX= comb[comb['class']!=7].iloc[:,65:69]
coldX= comb[comb['class']!=7].iloc[:,69:73]
diauX= comb[comb['class']!=7].iloc[:,73:80]
print(list(X.columns.values))
print(list(alphaX.columns.values))
print(list(eluX.columns.values))
print(list(edcX.columns.values))
print(list(spotX.columns.values))
print(list(spoX.columns.values))
print(list(heatX.columns.values))
print(list(dttX.columns.values))
print(list(coldX.columns.values))
print(list(diauX.columns.values))

['alpha 0', 'alpha 7', 'alpha 14', 'alpha 21', 'alpha 28', 'alpha 35', 'alpha 42', 'alpha 49', 'alpha 56', 'alpha 63', 'alpha 70', 'alpha 77', 'alpha 84', 'alpha 91', 'alpha 98', 'alpha 105', 'alpha 112', 'alpha 119', 'Elu 0', 'Elu 30', 'Elu 60', 'Elu 90', 'Elu 120', 'Elu 150', 'Elu 180', 'Elu 210', 'Elu 240', 'Elu 270', 'Elu 300', 'Elu 330', 'Elu 360', 'Elu 390', 'cdc15 10', 'cdc15 30', 'cdc15 50', 'cdc15 70', 'cdc15 90', 'cdc15 110', 'cdc15 130', 'cdc15 150', 'cdc15 170', 'cdc15 190', 'cdc15 210', 'cdc15 230', 'cdc15 250', 'cdc15 270', 'cdc15 290', 'spo 0', 'spo 2', 'spo 5', 'spo 7', 'spo 9', 'spo 11', 'spo5 2', 'spo5 7', 'spo5 11', 'spo- early', 'spo- mid', 'heat 0', 'heat 10', 'heat 20', 'heat 40', 'heat 80', 'heat 160', 'dtt 15', 'dtt 30', 'dtt 60', 'dtt 120', 'cold 0', 'cold 20', 'cold 40', 'cold 160', 'diau a', 'diau b', 'diau c', 'diau d', 'diau e', 'diau f', 'diau g']
['alpha 0', 'alpha 7', 'alpha 14', 'alpha 21', 'alpha 28', 'alpha 35', 'alpha 42', 'alpha 49', 'alpha 56', 'al

In [137]:
X = comb[comb['class']!=7].iloc[:,1:-1]
y = comb[comb['class']!=7].iloc[:,-1]

In [138]:
newX= comb[comb['class']!=7].iloc[:,:-1] #getting name

In [139]:
newX.head(5)
print(newX.shape)

(2464, 80)


In [140]:
tag_values=[]
docs=[]
i=0
for index, row in newX.iterrows():
 w=row["NAME"].split()
 t1=[i]
 docs.append(TaggedDocument(words=w, tags=t1))
 i+=1

In [141]:
size=15
model = models.Doc2Vec(vector_size=size, window=3, min_count=2, workers=4, epochs=20)
model.build_vocab(docs)

In [142]:
model.train(docs, total_examples=model.corpus_count, epochs=model.iter)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


In [143]:
namex=np.zeros((newX.shape[0], size))
for i in range(newX.shape[0]):
    namex[i]=model[i]

In [144]:
print(namex.shape)

(2464, 15)


In [145]:
from sklearn.preprocessing import Imputer

imputer = Imputer(missing_values = np.nan, strategy = 'mean', axis=0)
imputer = imputer.fit(X)
X_imputered=imputer.transform(X)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [146]:
print(X_imputered.shape)
print(namex.shape)

(2464, 79)
(2464, 15)


In [147]:
X_imputered=np.concatenate((namex,X_imputered), axis=1)

In [148]:
print(X_imputered.shape)
print(y.shape)

(2464, 94)
(2464,)


In [149]:
#saving data before sampling it, and after imputing it.
np.savetxt("newdata.txt", X_imputered, delimiter=" ")
np.savetxt("newlabels.txt", y, delimiter=" ")

### dealing imbalance
`over` and `under` are dictionary for how many sample we want for each class

In [8]:
from collections import Counter
print("Original:")
print(sorted(Counter(y).items()))

from imblearn.over_sampling import SMOTE
over = {1:300,2:300,3:300,4:300,5:300,6:300} #change here to set over sampling goal for those class
X_over, y_over = SMOTE(sampling_strategy = over).fit_resample(X_imputered, y)
print("after over sampling:")
print(sorted(Counter(y_over).items()))

under = {0:800} # change here for under sampling goal
from imblearn.under_sampling import RandomUnderSampler
X_under, y_under = RandomUnderSampler(sampling_strategy = under ).fit_resample(X_over, y_over)
print("after Under sampling:")
print(sorted(Counter(y_under).items()))


# from imblearn.combine import SMOTEENN
# smote_enn = SMOTEENN(sampling_strategy='all',smote=SMOTE(sampling_strategy = c))
# X_resampled, y_resampled = smote_enn.fit_resample(X_imputered, y)
# print(sorted(Counter(y_resampled).items()))



Original:
[(0, 2240), (1, 14), (2, 27), (3, 121), (4, 35), (5, 11), (6, 16)]
after over sampling:
[(0, 2240), (1, 300), (2, 300), (3, 300), (4, 300), (5, 300), (6, 300)]
after Under sampling:
[(0, 800), (1, 300), (2, 300), (3, 300), (4, 300), (5, 300), (6, 300)]
